In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing Libraries

In [3]:
import pandas as pd
import datetime
from scipy.spatial.distance import cdist
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

Reading customer file

In [4]:
df = pd.read_csv("/content/drive/My Drive/ColabFiles/AWCustomers.csv")
print(df.head())

   CustomerID Title FirstName MiddleName  LastName Suffix  \
0       21173   NaN      Chad          C      Yuan    NaN   
1       13249   NaN      Ryan        NaN     Perry    NaN   
2       29350   NaN     Julia        NaN  Thompson    NaN   
3       13503   NaN  Theodore        NaN     Gomez    NaN   
4       22803   NaN  Marshall          J      Shan    NaN   

             AddressLine1 AddressLine2         City    StateProvinceName  ...  \
0      7090 C. Mount Hood          NaN   Wollongong      New South Wales  ...   
1     3651 Willow Lake Rd          NaN      Shawnee     British Columbia  ...   
2  1774 Tice Valley Blvd.          NaN  West Covina           California  ...   
3         2103 Baldwin Dr          NaN    Liverpool              England  ...   
4         Am Gallberg 234          NaN        Werne  Nordrhein-Westfalen  ...   

         Education      Occupation Gender MaritalStatus HomeOwnerFlag  \
0        Bachelors        Clerical      M             M             1   


The reason to drop the following fields:

Customer Id: Unique key for each customer so its of no use in data mining.

Personal details like Name, Address, Phone number: Irrevelant as it identifies the customer but doesn't catergorize them.

City, State, Postal code: Many categories so on hot encoding will give many new features which will make model creation process slow and sometimes inaccurate.

Last updated: Date when record was created or updated, no relevance.

In [ ]:
irrevelant_fields = ['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1',
                     'AddressLine2', 'City', 'StateProvinceName', 'PostalCode', 'PhoneNumber', 'LastUpdated']
df1 = df.drop(irrevelant_fields, axis = 1)
cols = df1.select_dtypes(object).columns
df1[cols] = df1[cols].apply(lambda x: x.str.strip())
print(df1.head())

  CountryRegionName   BirthDate        Education      Occupation Gender  \
0         Australia  1987-11-13        Bachelors        Clerical      M   
1            Canada  1972-07-21  Partial College        Clerical      M   
2     United States  1985-11-09        Bachelors        Clerical      F   
3    United Kingdom  1977-10-18  Partial College  Skilled Manual      M   
4           Germany  1975-02-05  Partial College  Skilled Manual      M   

  MaritalStatus  HomeOwnerFlag  NumberCarsOwned  NumberChildrenAtHome  \
0             M              1                3                     0   
1             M              1                2                     1   
2             S              0                3                     0   
3             M              1                2                     1   
4             S              1                1                     0   

   TotalChildren  YearlyIncome  
0              1         81916  
1              2         81076  
2          

Creating new dataset with selected attributes

Converting birth date to age which will ease data preprocessing process

In [ ]:
def calculate_age(dob):
        dob = datetime.datetime.strptime(dob, "%Y-%m-%d")
        today = datetime.date.today()
        return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

df1['BirthDate'] = df['BirthDate'].apply(calculate_age)
df1 = df1.rename(columns = {"BirthDate" : "Age"})

Checking for null fields in dataframe.

In [ ]:
print(df1.isna().sum())

CountryRegionName       0
Age                     0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
dtype: int64


Normalizing Gender, Marital status, Education.

In [ ]:
gender = {"M": 0, "F": 1}
df1["Gender"] = df1["Gender"].map(gender)
married = {"S": 0, "M": 1}
df1["MaritalStatus"] = df1["MaritalStatus"].map(married)
education = {'Bachelors': 4, 'Partial College': 3, 'High School': 2, 'Graduate Degree': 5, 'Partial High School': 1}
df1['Education'] = df1['Education'].map(education)
print(df1.head())

  CountryRegionName  Age  Education      Occupation  Gender  MaritalStatus  \
0         Australia   36          4        Clerical       0              1   
1            Canada   51          3        Clerical       0              1   
2     United States   38          4        Clerical       1              0   
3    United Kingdom   46          3  Skilled Manual       0              1   
4           Germany   49          3  Skilled Manual       0              0   

   HomeOwnerFlag  NumberCarsOwned  NumberChildrenAtHome  TotalChildren  \
0              1                3                     0              1   
1              1                2                     1              2   
2              0                3                     0              0   
3              1                2                     1              2   
4              1                1                     0              0   

   YearlyIncome  
0         81916  
1         81076  
2         86387  
3         6148

One hot encoding Country and Occupation.

In [ ]:
enc = OneHotEncoder()
df_country = pd.DataFrame(enc.fit_transform(df1[["CountryRegionName"]]).toarray())
df_country.columns = enc.get_feature_names_out(['CountryRegionName'])
df1 = df1.drop(['CountryRegionName'], axis = 1)
df1 = pd.concat([df1, df_country], axis = 1)
df_occupation = pd.DataFrame(enc.fit_transform(df1[["Occupation"]]).toarray())
df_occupation.columns = enc.get_feature_names_out(['Occupation'])
df1 = df1.drop(['Occupation'], axis = 1)
df1 = pd.concat([df1, df_occupation], axis = 1)
print(df1.head())

   Age  Education  Gender  MaritalStatus  HomeOwnerFlag  NumberCarsOwned  \
0   36          4       0              1              1                3   
1   51          3       0              1              1                2   
2   38          4       1              0              0                3   
3   46          3       0              1              1                2   
4   49          3       0              0              1                1   

   NumberChildrenAtHome  TotalChildren  YearlyIncome  \
0                     0              1         81916   
1                     1              2         81076   
2                     0              0         86387   
3                     1              2         61481   
4                     0              0         51804   

   CountryRegionName_Australia  CountryRegionName_Canada  \
0                          1.0                       0.0   
1                          0.0                       1.0   
2                         

Performing standardization/Normalization on Age and Yearly Income.

In [ ]:
standard = StandardScaler()
minmax = MinMaxScaler()
df1[['Age', 'YearlyIncome']] = standard.fit_transform(df1[['Age', 'YearlyIncome']])
df1[['Education', 'Age', 'YearlyIncome', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren']] = minmax.fit_transform(df1[['Education', 'Age', 'YearlyIncome', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren']])

Now every feature in dataframe has values ranging between 0 and 1. Displaying preprocessed dataframe below.

In [ ]:
print(df1.head())

        Age  Education  Gender  MaritalStatus  HomeOwnerFlag  NumberCarsOwned  \
0  0.185714       0.75       0              1              1              0.6   
1  0.400000       0.50       0              1              1              0.4   
2  0.214286       0.75       1              0              0              0.6   
3  0.328571       0.50       0              1              1              0.4   
4  0.371429       0.50       0              0              1              0.2   

   NumberChildrenAtHome  TotalChildren  YearlyIncome  \
0              0.000000       0.333333      0.496842   
1              0.333333       0.666667      0.489453   
2              0.000000       0.000000      0.536172   
3              0.333333       0.666667      0.317083   
4              0.000000       0.000000      0.231958   

   CountryRegionName_Australia  CountryRegionName_Canada  \
0                          1.0                       0.0   
1                          0.0                       1.0

We could have also performed binning on Age and Yearly Income but it would have lead to some loss of data so it was better to standardize/normalize these features.

In [ ]:
#df1["Age"] = pd.cut(df1['Age'], bins = 7)
#df1["Age"] = df1["Age"].apply(lambda x: int(x.mid))
#df1['Age'] = df1['Age'].astype('int64')
#df1["YearlyIncome"] = pd.cut(df1['YearlyIncome'], bins = 7)
#df1["YearlyIncome"] = df1["YearlyIncome"].apply(lambda x: int(x.mid))
#df1['YearlyIncome'] = df1['YearlyIncome'].astype('int64')

Computing single matching coefficient, jaccard and cosine distance of first two objects in dataframe.

In [ ]:
smc = list(df1.loc[0] == df1.loc[1]).count(True)/len(df1.columns)
jac = cdist(df1.loc[0 : 1], df1.loc[0 : 1], metric = 'jaccard')[0][1]
cos = cdist(df1.loc[0 : 1], df1.loc[0 : 1], metric = 'cosine')[0][1]
print(smc, jac, cos)

0.6 0.7272727272727273 0.22196511398608987


Printing correlation value between Age and Yearly Income.

In [ ]:
print(df1['Age'].corr(df1['YearlyIncome']))

0.026223504178886616